# Task 1 (Detecting pathological gamblers)
### Using BoW methods with rolling window approach

In [1]:
P_FILE = "../posts.csv"
TRAIN_TOKEN="../train_df.csv"
TEST_TOKEN="../test_df.csv"
GENERAL_MODELS="../Models"
ROLLING_WINDOW_SIZE=10
TF_IDF_FOUND=True
BASELINE_COMP=False
TASK=1
#if sentiment:
#    MODEL_PATH =f"{GENERAL_MODELS}/BoW/TFIDF_win_{ROLLING_WINDOW_SIZE}_vader_tb" 
#else:
MODEL_PATH =f"{GENERAL_MODELS}/BoW/TFIDF_win_{ROLLING_WINDOW_SIZE}" 

BASIC_PATH=f"{GENERAL_MODELS}/BoW/TFIDF_win_{ROLLING_WINDOW_SIZE}" 

In [2]:
from pathlib import Path
Path(MODEL_PATH).mkdir(parents=True, exist_ok=True)

### opening the preprocessed dataset with pandas

In [3]:
import pandas as pd
import numpy as np
import os
seed=23
np.random.seed(seed)

train_df = pd.read_csv(TRAIN_TOKEN, sep='\t')
test_df = pd.read_csv(TEST_TOKEN, sep='\t')
train_df

,User,Post_Nr,Raw,Stemmed,Lemmatized,Label
0,3450,0,"sports betting number k in debt, feeling very ...","sport bet number k in debt , feel veri depress...","sports betting number k in debt , feeling very...",1
1,3450,1,finally accepted that you cannot win gambling ...,final accept that you can not win gambl relaps...,finally accepted that you can not win gambling...,1
2,3450,2,blocking software betfilter has anybody used t...,"block softwar betfilt has anybodi use this , w...",blocking software betfilter has anybody used t...,1
3,3450,3,prone to relapse when in debt? i find that whe...,prone to relaps when in debt ? i find that whe...,prone to relapse when in debt ? i find that wh...,1
4,3450,4,down to my last number on credit card i am num...,down to my last number on credit card i am num...,down to my last number on credit card i am num...,1
...,...,...,...,...,...,...
97853,162,670,you sick fuck,you sick fuck,you sick fuck,0
97854,162,671,reeeeeeeeeeeeeeeeeeeeeeeeeee,reeeeeeeeeeeeeeeeeeeeeeeeeee,reeeeeeeeeeeeeeeeeeeeeeeeeee,0
97855,162,672,this is so clearly satire! i really do not wan...,this is so clear satir ! i realli do not want ...,this is so clearly satire ! i really do not wa...,0
97856,162,673,he cannot write a story in number minutes,he can not write a stori in number minut,he can not write a story in number minutes,0


In [4]:
def rolling_window(df, window_size,stride, field):
    res_map={}
    for user in df['User'].unique():
        user_df = df[df['User']==user]
        res_map[user]=(user_df['Label'].values[0],{})
        posts = user_df[field].values
        iteration=0
        for i in range(0,len(posts),stride):
            res_map[user][1][iteration]=' '.join((posts[i:i+window_size]))
            iteration+=1
    result_df = pd.DataFrame([(k,k1,v1,v[0]) for k,v in res_map.items() for k1,v1 in v[1].items()], columns = ['User','Window_id','Text','Label'])
    
    return result_df

In [5]:
train_df = rolling_window(train_df,ROLLING_WINDOW_SIZE,1,'Lemmatized')
test_df = rolling_window(test_df,ROLLING_WINDOW_SIZE,1,'Lemmatized')

train_df

,User,Window_id,Text,Label
0,3450,0,"sports betting number k in debt , feeling very...",1
1,3450,1,finally accepted that you can not win gambling...,1
2,3450,2,blocking software betfilter has anybody used t...,1
3,3450,3,prone to relapse when in debt ? i find that wh...,1
4,3450,4,down to my last number on credit card i am num...,1
...,...,...,...,...
97853,162,670,you sick fuck reeeeeeeeeeeeeeeeeeeeeeeeeee thi...,0
97854,162,671,reeeeeeeeeeeeeeeeeeeeeeeeeee this is so clearl...,0
97855,162,672,this is so clearly satire ! i really do not wa...,0
97856,162,673,he can not write a story in number minutes pay...,0


In [6]:
test_df

,User,Window_id,Text,Label
0,6037,0,number with low self esteem . honest opinions ...,1
1,6037,1,i do not think you need to do anything to make...,1
2,6037,2,laughing out loud what is with those grandpa s...,1
3,6037,3,i think you are really good looking . i agree ...,1
4,6037,4,i have number tickets to the first night at ha...,1
...,...,...,...,...
22318,2517,59,"who are you , who are so wise in the ways of s...",0
22319,2517,60,"that would sad man , take my updoot f the tric...",0
22320,2517,61,f the trickster doctor,0
22321,2517,62,the trickster doctor,0


In [7]:
train_df = pd.concat([train_df,test_df])

In [8]:
    #test_df.drop(['VADER','TB'], inplace=True, axis=1)


In [9]:
"""
if sentiment:
    test_df['TB'] = test_df['Text'].apply(lambda text: TextBlob(text).sentiment)
    test_df['VADER'] = test_df['Text'].apply(lambda text: sia.polarity_scores(text))
    test_df['polarity'] = test_df['TB'].apply(lambda tb: tb[0])
    test_df['subjectivity'] = test_df['TB'].apply(lambda tb: tb[1])
    test_df['negativity'] = test_df['VADER'].apply(lambda v: v['neg'])
    test_df['positivity'] = test_df['VADER'].apply(lambda v: v['pos'])
    test_df['neutrality'] = test_df['VADER'].apply(lambda v: v['neu'])
    test_df['compound'] = test_df['VADER'].apply(lambda v: v['compound'])

    test_df
"""

"\nif sentiment:\n    test_df['TB'] = test_df['Text'].apply(lambda text: TextBlob(text).sentiment)\n    test_df['VADER'] = test_df['Text'].apply(lambda text: sia.polarity_scores(text))\n    test_df['polarity'] = test_df['TB'].apply(lambda tb: tb[0])\n    test_df['subjectivity'] = test_df['TB'].apply(lambda tb: tb[1])\n    test_df['negativity'] = test_df['VADER'].apply(lambda v: v['neg'])\n    test_df['positivity'] = test_df['VADER'].apply(lambda v: v['pos'])\n    test_df['neutrality'] = test_df['VADER'].apply(lambda v: v['neu'])\n    test_df['compound'] = test_df['VADER'].apply(lambda v: v['compound'])\n\n    test_df\n"

In [10]:

train_df = train_df.sample(frac=1, random_state=seed).reset_index(drop=True) 
X_train = train_df['Text']


### Since we are starting the actual training, here are some utility functions which we will use to save our trained models.

In [11]:
import pickle
def save_model(model, name):
    with open(f"{MODEL_PATH}/{name}",'wb') as f:
        pickle.dump( model,f)

### Here we select the stopwords to be considered when building the tf-idf matrix

In [12]:
from nltk.corpus import stopwords
nltk_stop_words = set(stopwords.words("english"))
import gensim
gensim_stop_words = gensim.parsing.preprocessing.STOPWORDS

In [13]:
nltk_stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [14]:
gensim_stop_words

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

### Before moving on to the model section, we will assess which parameters to use in the tfidfvectorizer by performing grid search (since it is a fast process with a Naive Bayes algorithm as the final classifier)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
#from scipy.stats import uniform,  randint
"""
parameters = {
    'max_features':[5000,7500,10000,12500,15000],
    'max_df': [0.25,0.5,0.75,1.0],
    'stop_words':['english',nltk_stop_words,gensim_stop_words],
    'ngram_range':[(1,1),(1,2)]
}

"""



"\nparameters = {\n    'max_features':[5000,7500,10000,12500,15000],\n    'max_df': [0.25,0.5,0.75,1.0],\n    'stop_words':['english',nltk_stop_words,gensim_stop_words],\n    'ngram_range':[(1,1),(1,2)]\n}\n\n"

In [16]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

def tune_vectorizer(df,params, n_folds=5):
    skf = StratifiedKFold(n_splits=n_folds)
    user_label_df =df.drop_duplicates('User')
    users = user_label_df['User'].to_numpy()
    
    labels = user_label_df['Label'].to_numpy()
    #print(labels)
    #print(users.shape,labels.shape)
    model = MultinomialNB()
    f1_scores = []
    for train_index, test_index in skf.split(users, labels):
        transformer = TfidfVectorizer(**params)
        train_users = [users[f] for f in train_index]
        test_users = [users[f] for f in test_index]

        train_folds = df[df['User'].isin(train_users)]
        X_train = transformer.fit_transform(train_folds['Text'])
        test_folds = df[df['User'].isin(test_users)]
        X_test = transformer.transform(test_folds['Text'])
       

        
        
        model.fit(X_train, train_folds['Label'])
        f1_scores.append(f1_score(test_folds['Label'],model.predict(X_test)))

    return f1_scores

In [17]:
sw_lists = {'english':'english','nltk_stop_words':nltk_stop_words,'gensim_stop_words':gensim_stop_words}

In [18]:
def objective(trial):
    parameters = {
        'max_features':trial.suggest_int('max_features',1000,15000,step=1000),
        'max_df': trial.suggest_float('max_df',0.20,1.0,step=0.05),
        'stop_words':trial.suggest_categorical('stop_words',['english','nltk_stop_words','gensim_stop_words']),
        'ngram_range':trial.suggest_int('ngram_range',1,3),
        'sublinear_tf':trial.suggest_int('sublinear_tf',0,1)
        
        
    }
    
  
    
    avg_f1 = train_eval(trial,parameters,5,train_df)
    return np.mean(avg_f1)

In [19]:
from tqdm import tqdm
from sklearn.metrics import f1_score

def train_eval(trial,params, n_folds, df):

    skf = StratifiedKFold(n_splits=n_folds)
    user_label_df =df.drop_duplicates('User')
    users = user_label_df['User'].to_numpy()
    
    labels = user_label_df['Label'].to_numpy()

    #format the vectorizer can accept
    params['stop_words'] = sw_lists[params['stop_words']]
    params['ngram_range'] = (1,params['ngram_range'])
    
    f1_scores = []
    for fold,(train_index, test_index) in enumerate(skf.split(users, labels)):
        transformer = TfidfVectorizer(**params)
        train_users = [users[f] for f in train_index]
        test_users = [users[f] for f in test_index]

        train_folds = df[df['User'].isin(train_users)]
        X_train = transformer.fit_transform(train_folds['Text'])
        test_folds = df[df['User'].isin(test_users)]
        X_test = transformer.transform(test_folds['Text'])
    
        model = MultinomialNB()
        model.fit(X_train, train_folds['Label'])
        f1_scores.append(f1_score(test_folds['Label'],model.predict(X_test)))
    
        trial.report(np.mean(f1_scores), fold)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return f1_scores


In [20]:
import optuna
import joblib
if not TF_IDF_FOUND:
    study = optuna.create_study(
        study_name=f"t{TASK}_tfidfvec_{ROLLING_WINDOW_SIZE}", storage=f"mysql://root@localhost:3306/t{TASK}_tfidfvec_{ROLLING_WINDOW_SIZE}",
        load_if_exists=True, direction='maximize')
    joblib.dump(study,f"t{TASK}_tfidfvec_{ROLLING_WINDOW_SIZE}")
    #study = optuna.create_study(study_name=f"tfidfvectorizer_cv_{ROLLING_WINDOW_SIZE}",direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.NopPruner())
    #study.optimize(objective, n_trials=30, timeout=(60*60*24))
else:
    study = joblib.load(f"study_cv_tfidfvectorizer_{ROLLING_WINDOW_SIZE}.pkl")

In [21]:

#study = joblib.load("study.pkl")
print("Best trial until now:")
print(" Value: ", study.best_trial.value)
print(" Params: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")


Best trial until now:
 Value:  0.820679
 Params: 
    max_df: 0.6
    max_features: 15000
    ngram_range: 2
    stop_words: nltk_stop_words
    sublinear_tf: 1


In [22]:
fig = optuna.visualization.plot_param_importances(study)
fig.write_image(f"tfidf_feature_importance_{ROLLING_WINDOW_SIZE}.png")
fig.show()

In [23]:
fig2=optuna.visualization.plot_parallel_coordinate(study)
fig2.write_image(f"tfidf_parallelc_{ROLLING_WINDOW_SIZE}.png")
fig2.show()

In [24]:
fig3=optuna.visualization.plot_slice(study)
fig3.write_image(f"tfidf_plotslice_{ROLLING_WINDOW_SIZE}.png")
fig3.show()

In [26]:
tfidf_params = study.best_trial.params
tfidf_params['stop_words'] = sw_lists[tfidf_params['stop_words']]
tfidf_params['ngram_range'] = (1,tfidf_params['ngram_range'])


In [27]:
"""
if sentiment:
    from scipy.sparse import hstack
    X_train_transformed = hstack((X_train_transformed,np.array(train_df['polarity'])[:,None]))
    X_train_transformed = hstack((X_train_transformed,np.array(train_df['subjectivity'])[:,None]))
    X_train_transformed = hstack((X_train_transformed,np.array(train_df['negativity'])[:,None]))
    X_train_transformed = hstack((X_train_transformed,np.array(train_df['positivity'])[:,None]))
    X_train_transformed = hstack((X_train_transformed,np.array(train_df['neutrality'])[:,None]))
    X_train_transformed = hstack((X_train_transformed,np.array(train_df['compound'])[:,None]))

    X_test_transformed = hstack((X_test_transformed,np.array(test_df['polarity'])[:,None]))
    X_test_transformed = hstack((X_test_transformed,np.array(test_df['subjectivity'])[:,None]))
    X_test_transformed = hstack((X_test_transformed,np.array(test_df['negativity'])[:,None]))
    X_test_transformed = hstack((X_test_transformed,np.array(test_df['positivity'])[:,None]))
    X_test_transformed = hstack((X_test_transformed,np.array(test_df['neutrality'])[:,None]))
    X_test_transformed = hstack((X_test_transformed,np.array(test_df['compound'])[:,None]))
"""
    



"\nif sentiment:\n    from scipy.sparse import hstack\n    X_train_transformed = hstack((X_train_transformed,np.array(train_df['polarity'])[:,None]))\n    X_train_transformed = hstack((X_train_transformed,np.array(train_df['subjectivity'])[:,None]))\n    X_train_transformed = hstack((X_train_transformed,np.array(train_df['negativity'])[:,None]))\n    X_train_transformed = hstack((X_train_transformed,np.array(train_df['positivity'])[:,None]))\n    X_train_transformed = hstack((X_train_transformed,np.array(train_df['neutrality'])[:,None]))\n    X_train_transformed = hstack((X_train_transformed,np.array(train_df['compound'])[:,None]))\n\n    X_test_transformed = hstack((X_test_transformed,np.array(test_df['polarity'])[:,None]))\n    X_test_transformed = hstack((X_test_transformed,np.array(test_df['subjectivity'])[:,None]))\n    X_test_transformed = hstack((X_test_transformed,np.array(test_df['negativity'])[:,None]))\n    X_test_transformed = hstack((X_test_transformed,np.array(test_df['po

### Now we test the baseline performance of some typical models, using sklearn's default parameters

In [28]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from scipy.sparse import hstack


def custom_cv(model, df, n_folds=5,sent =False):
    skf = StratifiedKFold(n_splits=n_folds)
    user_label_df =df.drop_duplicates('User')
    users = user_label_df['User'].to_numpy()
    
    labels = user_label_df['Label'].to_numpy()
    #print(labels)
    #print(users.shape,labels.shape)
    
    f1_scores = []
    for train_index, test_index in skf.split(users, labels):
        vectorizer = TfidfVectorizer(**tfidf_params)

        train_users = [users[f] for f in train_index]
        test_users = [users[f] for f in test_index]

        train_folds = df[df['User'].isin(train_users)].copy()
        X_train = vectorizer.fit_transform(train_folds['Text'])
        test_folds = df[df['User'].isin(test_users)].copy()
        X_test = vectorizer.transform(test_folds['Text'])
        
        
        if sent:
            
            X_train = hstack((X_train,np.array(train_folds['polarity'])[:,None]))
            X_train = hstack((X_train,np.array(train_folds['subjectivity'])[:,None]))
            X_train = hstack((X_train,np.array(train_folds['negativity'])[:,None]))
            X_train = hstack((X_train,np.array(train_folds['positivity'])[:,None]))
            X_train = hstack((X_train,np.array(train_folds['neutrality'])[:,None]))
            X_train = hstack((X_train,np.array(train_folds['compound'])[:,None]))

            X_test = hstack((X_test,np.array(test_folds['polarity'])[:,None]))
            X_test = hstack((X_test,np.array(test_folds['subjectivity'])[:,None]))
            X_test = hstack((X_test,np.array(test_folds['negativity'])[:,None]))
            X_test = hstack((X_test,np.array(test_folds['positivity'])[:,None]))
            X_test = hstack((X_test,np.array(test_folds['neutrality'])[:,None]))
            X_test = hstack((X_test,np.array(test_folds['compound'])[:,None]))
            
        

        
        
        model.fit(X_train, train_folds['Label'])
        f1_scores.append(f1_score(test_folds['Label'],model.predict(X_test)))

    return f1_scores

In [29]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.ensemble import ExtraTreesClassifier
if BASELINE_COMP:
        models = {
                'sgdLR':SGDClassifier(random_state=seed,loss='log'),
                'NB':MultinomialNB(),\
                'sgdlSVM':SGDClassifier(random_state=seed,loss='hinge'),
                'ExtraTrees':ExtraTreesClassifier(random_state=seed,n_jobs=-1),\
                'Perceptron':Perceptron(random_state=seed)}
        report=""
        best_model_name = ""
        best_model=None
        best_f1=0
        for model_name, model in models.items():
                res = custom_cv(model,train_df)
                if np.mean(res) > best_f1:
                        best_f1=np.mean(res)
                        best_model_name=model_name
                        best_model=model
                report+=f"{model_name} f1: {round(np.mean(res),3)}\n"
                print(f"{model_name} f1: {round(np.mean(res),3)}")
        #cv
        with open(f"{MODEL_PATH}/cv_baseline_report.txt",'w') as f:
                f.write(report)

In [30]:
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer

if BASELINE_COMP:
    sia = SentimentIntensityAnalyzer()
    train_df['TB'] = train_df['Text'].apply(lambda text: TextBlob(text).sentiment)
    train_df['VADER'] = train_df['Text'].apply(lambda text: sia.polarity_scores(text))

    train_df['polarity'] = train_df['TB'].apply(lambda tb: (tb[0]+1/2))
    train_df['subjectivity'] = train_df['TB'].apply(lambda tb: tb[1])
    train_df['negativity'] = train_df['VADER'].apply(lambda v: v['neg'])
    train_df['positivity'] = train_df['VADER'].apply(lambda v: v['pos'])
    train_df['neutrality'] = train_df['VADER'].apply(lambda v: v['neu'])
    train_df['compound'] = train_df['VADER'].apply(lambda v: (v['compound']+1)/2)



    train_df.drop(['VADER','TB'], inplace=True, axis=1)


In [31]:
if BASELINE_COMP:
        res = custom_cv(best_model,train_df, sent=True)
        print(f"{best_model_name} f1: {round(np.mean(res),3)}")
        #cv
        with open(f"{MODEL_PATH}/cv_baseline_report.txt",'a') as f:
                f.write(f"\nBest model with sent:\n{best_model_name} f1: {round(np.mean(res),3)}")

## Hyper parameter tuning

#### Best performing model was sgd Logistic Regression for window size 10 without SA features, having achieved a 0.823 F1 score

In [33]:
from tqdm import tqdm

def train_eval_tuning(trial,params, df):

    skf = StratifiedKFold(n_splits=5)
    user_label_df =df.drop_duplicates('User')
    users = user_label_df['User'].to_numpy()
    
    labels = user_label_df['Label'].to_numpy()

    
    f1_scores = []
    for fold,(train_index, test_index) in enumerate(skf.split(users, labels)):
        transformer = TfidfVectorizer(**tfidf_params)
        train_users = [users[f] for f in train_index]
        test_users = [users[f] for f in test_index]

        train_folds = df[df['User'].isin(train_users)]
        X_train = transformer.fit_transform(train_folds['Text'])
        test_folds = df[df['User'].isin(test_users)]
        X_test = transformer.transform(test_folds['Text'])
        model = SGDClassifier(**params)
    
        model.fit(X_train, train_folds['Label'])
        f1_scores.append(f1_score(test_folds['Label'],model.predict(X_test)))
    
        trial.report(np.mean(f1_scores), fold)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return f1_scores


In [36]:
def tuning_objective(trial):
    parameters = {
        'max_iter':trial.suggest_int('max_iter',1000,2500,step=500),
        'loss':trial.suggest_categorical('loss',['log']),
        'penalty':trial.suggest_categorical('penalty',['l2','l1','elasticnet']),
        'alpha': trial.suggest_float('alpha',0.00001,0.1,log=True),
        'random_state':trial.suggest_int('random_state',seed,seed)
        
        
    }
    
  
    
    avg_f1 = train_eval_tuning(trial,parameters,train_df)
    return np.mean(avg_f1)

In [37]:
study = optuna.create_study(
        study_name=f"t{TASK}_tuning_{ROLLING_WINDOW_SIZE}",
        direction='maximize')
study.optimize(tuning_objective, n_trials=30, timeout=(60*60*24))
joblib.dump(study,f"t{TASK}_tuning_{ROLLING_WINDOW_SIZE}.pkl")
    #study = optuna.create_study(study_name=f"tfidfvectorizer_cv_{ROLLING_WINDOW_SIZE}",direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.NopPruner())
study = joblib.load(f"t{TASK}_tuning_{ROLLING_WINDOW_SIZE}.pkl")

[I 2022-06-13 08:38:40,913] A new study created in memory with name: t1_tuning_10
[I 2022-06-13 08:42:15,843] Trial 0 finished with value: 0.0 and parameters: {'max_iter': 2000, 'loss': 'log', 'penalty': 'l1', 'alpha': 0.056923292859782745, 'random_state': 23}. Best is trial 0 with value: 0.0.
[I 2022-06-13 08:45:54,446] Trial 1 finished with value: 0.8073830828248536 and parameters: {'max_iter': 2000, 'loss': 'log', 'penalty': 'elasticnet', 'alpha': 0.0003879236418669762, 'random_state': 23}. Best is trial 1 with value: 0.8073830828248536.
[I 2022-06-13 08:49:29,735] Trial 2 finished with value: 0.0 and parameters: {'max_iter': 2500, 'loss': 'log', 'penalty': 'l1', 'alpha': 0.030937375373866316, 'random_state': 23}. Best is trial 1 with value: 0.8073830828248536.
[I 2022-06-13 08:53:04,888] Trial 3 finished with value: 0.0 and parameters: {'max_iter': 2000, 'loss': 'log', 'penalty': 'l1', 'alpha': 0.008449699964826179, 'random_state': 23}. Best is trial 1 with value: 0.807383082824853

In [38]:
study.optimize(tuning_objective, n_trials=20, timeout=(60*60*24))


[I 2022-06-13 10:14:33,605] Trial 30 finished with value: 0.8227151622100239 and parameters: {'max_iter': 2000, 'loss': 'log', 'penalty': 'l2', 'alpha': 0.00011563033578742476, 'random_state': 23}. Best is trial 15 with value: 0.8241696050176378.
[I 2022-06-13 10:18:12,379] Trial 31 finished with value: 0.8236657366718058 and parameters: {'max_iter': 2500, 'loss': 'log', 'penalty': 'elasticnet', 'alpha': 6.598464965587579e-05, 'random_state': 23}. Best is trial 15 with value: 0.8241696050176378.
[I 2022-06-13 10:21:51,206] Trial 32 finished with value: 0.8231545956682538 and parameters: {'max_iter': 2500, 'loss': 'log', 'penalty': 'elasticnet', 'alpha': 7.632380579780299e-05, 'random_state': 23}. Best is trial 15 with value: 0.8241696050176378.
[I 2022-06-13 10:23:57,978] Trial 33 pruned. 
[I 2022-06-13 10:24:38,278] Trial 34 pruned. 
[I 2022-06-13 10:25:19,105] Trial 35 pruned. 
[I 2022-06-13 10:25:59,281] Trial 36 pruned. 
[I 2022-06-13 10:26:40,177] Trial 37 pruned. 
[I 2022-06-13 1

In [39]:
joblib.dump(study,f"t{TASK}_tuning_{ROLLING_WINDOW_SIZE}.pkl")


['t1_tuning_10.pkl']

In [40]:
sgdLR_params = study.best_trial.params
sgdLR_params

{'max_iter': 2500,
 'loss': 'log',
 'penalty': 'l2',
 'alpha': 6.298583165702807e-05,
 'random_state': 23}

In [41]:
final_vectorizer = TfidfVectorizer(**tfidf_params)
final_vectorizer.fit(train_df['Text'])
final_model = SGDClassifier(**sgdLR_params)
final_model.fit(final_vectorizer.transform(train_df['Text']), train_df['Label'])
save_model(final_vectorizer, "optimized_tfidf.pkl")
save_model(final_model, "optimized_sgdLR.pkl")
